In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestRegressor


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### Data preprocessing

In [ ]:
data = pd.read_csv('combined_db.csv', skiprows=[0,2])

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data[data['bsf stage (nursery or fattening)']=='fattening']['Diet number \n(from the file with the diets n_ for nursery and f_ for fattening)'].nunique()

In [ ]:
without_na = data.dropna(subset=[' total larval  biomass, gr','Average larval  mass, mg','Fat, %', 'Recalculated protein, %'])
without_na[without_na['bsf stage (nursery or fattening)']=='fattening']['Diet number \n(from the file with the diets n_ for nursery and f_ for fattening)'].nunique()

In [ ]:
data[data['bsf stage (nursery or fattening)']=='nursery']['Diet number \n(from the file with the diets n_ for nursery and f_ for fattening)'].nunique()

In [ ]:
data = data.dropna(subset=['Larvae density'])

In [ ]:
data.loc[data['Larvae density'] == '12.5K', 'Larvae density'] = 12500

In [ ]:
data.loc[data['Larvae density'] == '12,500', 'Larvae density'] = 12500

In [ ]:
data.loc[data['Larvae density'] == '25,000', 'Larvae density'] = 25000

In [ ]:
data['Larvae density'] = data['Larvae density'].astype('int')

In [ ]:
data = data[data[' total larval  biomass, gr']!="N.A"]
data = data[data[' total larval  biomass, gr']!='#DIV/0!']
data = data[data[' total larval  biomass, gr']!='#VALUE!']

In [ ]:
data = data[data['Average larval  mass, mg']!='#VALUE!']

In [ ]:
data.head()

In [ ]:
columns = ['Dry Matter, %','Diet number \n(from the file with the diets n_ for nursery and f_ for fattening)','bsf stage (nursery or fattening)','Cycle, days','Initial Moisture', 'Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %','company_name',' total larval  biomass, gr','Average larval  mass, mg','Fat, %','Recalculated protein, %',
      "experiment purpose",'Survival%','Larvae density','start date (larvae t = 0 )','batch number '] #,'% of waste reduction (FW/FW)', '% of waste reduction (DM/DM)']
    #  'Feed conversion efficiency (DM/FW) without nursery',   
    #'Feed conversion efficiency (DM/FW) with nursery',
       #'Feed conversion efficiency (FW/FW) without nursery',
       #'Feed conversion efficiency (FW/FW) with nursery']

In [ ]:
data = data[data["experiment purpose"]!='larvae production']

In [ ]:
value_to_find ='לא סנין'
  #'לא סנין' # 'N.A' #'#VALUE!'#  'לא למלא, מחושב' 

for column in data.columns:
    if value_to_find in data[column].values:
        print(f"Value '{value_to_find}' found in column '{column}'")

In [ ]:
rows_to_remove = data[data.isin([value_to_find]).any(axis=1)]

data = data.drop(rows_to_remove.index)

In [ ]:
value_to_find ='לא למלא, מחושב'
  #'לא סנין' # 'N.A' #'#VALUE!'#  'לא למלא, מחושב' 

for column in data.columns:
    if value_to_find in data[column].values:
        print(f"Value '{value_to_find}' found in column '{column}'")

In [ ]:
rows_to_remove = data[data.isin([value_to_find]).any(axis=1)]

data = data.drop(rows_to_remove.index)

In [ ]:
value_to_find = 'שק מתאריך 04/04' 
for column in data.columns:
    if value_to_find in data[column].values:
        print(f"Value '{value_to_find}' found in column '{column}'")

In [ ]:
rows_to_remove = data[data.isin([value_to_find]).any(axis=1)]

data = data.drop(rows_to_remove.index)

In [ ]:
data[[ 'Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %',' total larval  biomass, gr','Average larval  mass, mg']] = data[['Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %',' total larval  biomass, gr','Average larval  mass, mg']].replace(u'\xa0', u'0').astype("float")

In [ ]:
data['Larvae density'].unique()

In [ ]:
data = data.rename(columns={'Diet number \n(from the file with the diets n_ for nursery and f_ for fattening)': 'Diet_num'})

### Dates processing

In [ ]:
data['date'] = pd.to_datetime(data['start date (larvae t = 0 )'] , dayfirst=True)

data['month'] = data['date'].dt.month
data['day_of_week'] = data['date'].dt.weekday  # Monday=0, Sunday=6

### Fattening

In [ ]:
data_fattening =data[data['bsf stage (nursery or fattening)']=="fattening"]

In [ ]:
#drop all rows with nan values for target variables
data_fattening = data_fattening.dropna(subset = [' total larval  biomass, gr','Average larval  mass, mg','Fat, %', 'Recalculated protein, %'])

### Nursery

In [ ]:
data_nursery =data[data['bsf stage (nursery or fattening)']=="nursery"]

In [ ]:
#drop all rows with nan values for target variables
data_nursery = data_nursery.dropna(subset = [' total larval  biomass, gr','Average larval  mass, mg'])

### Isolate diet 500

In [ ]:
data_fattening_without_500 = data_fattening[data_fattening["Diet_num"]!='500']

In [ ]:
data_fattening_without_500 = data_fattening[data_fattening["Diet_num"]!= 500]

In [ ]:
data_fattening_without_500 = data_fattening_without_500[data_fattening["Diet_num"]!='f_500']

In [ ]:
value_to_find = '#VALUE!' 
  #'לא סנין' # 'N.A' #'#VALUE!'#

for column in data_fattening_without_500.columns:
    if value_to_find in data_fattening_without_500[column].values:
        print(f"Value '{value_to_find}' found in column '{column}'")

In [ ]:
rows_to_remove = data_fattening_without_500[data_fattening_without_500.isin([value_to_find]).any(axis=1)]

# Remove rows containing the value
data_fattening_without_500 = data_fattening_without_500.drop(rows_to_remove.index)

In [ ]:
data_fattening_without_500[[' total larval  biomass, gr','Average larval  mass, mg','Fat, %','Recalculated protein, %']]=data_fattening_without_500[[' total larval  biomass, gr','Average larval  mass, mg','Fat, %','Recalculated protein, %']].replace(u'\xa0', u'0').astype('float')

In [ ]:
data_fattening_500 = data_fattening[(data_fattening["Diet_num"] == '500') | (data_fattening["Diet_num"] == 'f_500')]

In [ ]:
data_nursery_500 = data_nursery[(data_nursery["Diet_num"] == '500') | (data_nursery["Diet_num"] == 'n_500')]

In [ ]:
data_nursery_without_500 = data_nursery[data_nursery["Diet_num"]!='n_500']

In [ ]:
data_nursery_without_500 = data_nursery[data_nursery["Diet_num"]!='500']

In [ ]:
data_nursery_without_500 = data_nursery[data_nursery["Diet_num"]!=500]

In [ ]:
data_nursery_without_500.columns

In [ ]:
data_nursery_without_500[[ ' total larval  biomass, gr',
       'Average larval  mass, mg']] = data_nursery_without_500[[ ' total larval  biomass, gr',
       'Average larval  mass, mg']].astype('float')

In [ ]:
sns.histplot(data_fattening_without_500[['Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %']], kde=True)
plt.show()

In [ ]:
sns.histplot(data_nursery_without_500[['Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %']], kde=True)
plt.show()

### Fattening statistics

In [ ]:
data_fattening_without_500['Fat, %'] = data_fattening_without_500['Fat, %'].astype('float')

In [ ]:
sns.histplot(data_fattening_without_500['Fat, %'])
plt.show()

In [ ]:
data_fattening_without_500['Recalculated protein, %'] = data_fattening_without_500['Recalculated protein, %'].astype('float')

In [ ]:
sns.histplot(data_fattening_without_500['Recalculated protein, %'])
plt.show()

In [ ]:
data_fattening_without_500 = data_fattening_without_500[data_fattening_without_500['Recalculated protein, %']>0]

In [ ]:
sns.histplot(data_fattening_without_500[' total larval  biomass, gr'])
plt.show()

In [ ]:
sns.histplot(data_fattening_without_500['Average larval  mass, mg'])
plt.show()

In [ ]:
data_fattening_without_500 = data_fattening_without_500[data_fattening_without_500['Average larval  mass, mg']>0]

In [ ]:
data_fattening_without_500 = data_fattening_without_500[data_fattening_without_500[' total larval  biomass, gr']>0]

### Nursery without 500 statistics

In [ ]:
data_nursery_without_500['Average larval  mass, mg'] = data_nursery_without_500['Average larval  mass, mg'].astype('float')

In [ ]:
sns.histplot(data_nursery_without_500['Average larval  mass, mg'])
plt.show()

In [ ]:
sns.histplot(data_nursery_without_500[' total larval  biomass, gr'])
plt.show()

In [ ]:
data_nursery_without_500['Cycle, days'] = data_nursery_without_500['Cycle, days'].astype('float')

In [ ]:
data_nursery_without_500['Cycle, days'].describe()

In [ ]:
sns.histplot(data_nursery_without_500['Cycle, days'])
plt.show()

In [ ]:
data_fattening_without_500[['Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %']] = data_fattening_without_500[['Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %']].astype('float')

In [ ]:
data_fattening_without_500[[' total larval  biomass, gr','Average larval  mass, mg','Fat, %', 'Recalculated protein, %']] = data_fattening_without_500[[' total larval  biomass, gr','Average larval  mass, mg','Fat, %', 'Recalculated protein, %']].replace(u'\xa0', u'0').astype('float')

In [ ]:
data_fattening_without_500['Cycle, days'] = data_fattening_without_500['Cycle, days'].astype('float')

In [ ]:
sns.scatterplot(data=data_fattening_without_500, x='day_of_week', y=' total larval  biomass, gr')

In [ ]:
sns.scatterplot(data=data_fattening_without_500, x='Fat, \n%', y='Fat, %')

In [ ]:
data_fattening_without_500_run = data_fattening_without_500.copy()

In [ ]:

# Calculate the correlation matrix
corr_matrix = data_fattening_without_500[['Dry Matter, %','Larvae density','day_of_week','month','Cycle, days','Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %',' total larval  biomass, gr','Average larval  mass, mg','Fat, %','Recalculated protein, %']].corr()

mask = np.abs(corr_matrix) <= 0.5
# Create a heatmap to visualize the correlation matrix
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm',mask=mask)
plt.show()

In [ ]:
# Calculate the correlation matrix
corr_matrix = data_nursery_without_500[['Dry Matter, %','Larvae density','day_of_week','month','Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','NDF, %', 'Ash, %',' total larval  biomass, gr','Average larval  mass, mg']].corr()
mask = np.abs(corr_matrix) <= 0.5
# Create a heatmap to visualize the correlation matrix
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm',mask=mask)
plt.show()

In [ ]:
sns.scatterplot(data=data_fattening_without_500, x='Cycle, days', y='Fat, %')

# Find the x value for which y is maximum
x_max = data_fattening_without_500['Cycle, days'][data_fattening_without_500['Fat, %'].idxmax()]

# Add a vertical line at x_max
plt.axvline(x=x_max, color='r', linestyle='--')

# Show the plot
plt.show()

In [ ]:
sns.scatterplot(data=data_fattening_without_500, x='Cycle, days', y='Recalculated protein, %')

# Find the top 5 maximum y values and store them in a new DataFrame
top_5_max_points_df = data_fattening_without_500.nlargest(10, 'Recalculated protein, %')

# Sort the DataFrame by 'Cycle, days' to ensure the line is plotted correctly
top_5_max_points_df = top_5_max_points_df.sort_values('Cycle, days')

# Plot a line connecting the top 5 maximum points
plt.plot(top_5_max_points_df['Cycle, days'], top_5_max_points_df['Recalculated protein, %'], color='r', linestyle='--')

plt.show()

In [ ]:
sns.scatterplot(data=data_fattening_without_500, x='Cycle, days', y='Recalculated protein, %')

# Find the x value for which y is maximum
x_max = data_fattening_without_500['Cycle, days'][data_fattening_without_500['Recalculated protein, %'].idxmax()]

# Add a vertical line at x_max
plt.axvline(x=x_max, color='r', linestyle='--')

# Show the plot
plt.show()

In [ ]:
sns.kdeplot(data=data_fattening_without_500['Fat, %'], bw_adjust=0.5)
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

# Scatterplot
sns.scatterplot(data=data_fattening_without_500, x='Cycle, days', y='Recalculated protein, %', ax=ax1)

# Find the x value for which y is maximum
x_max = data_fattening_without_500['Cycle, days'][data_fattening_without_500['Recalculated protein, %'].idxmax()]

# Add a vertical line at x_max
ax1.axvline(x=x_max, color='r', linestyle='--')

# Create a second y-axis for the KDE plot
ax2 = ax1.twinx()

# KDE plot with adjusted bandwidth
sns.kdeplot(data=data_fattening_without_500['Cycle, days'], bw_adjust=0.4, color='g', ax=ax2)

# Show the plot
plt.show()

In [ ]:
sns.jointplot(data=data_fattening_without_500, x='Cycle, days', y='Fat, %', kind='kde')

In [ ]:
sns.jointplot(data=data_fattening_without_500, x='Cycle, days', y='Recalculated protein, %', kind='kde')

In [ ]:
sns.jointplot(data=data_fattening_without_500, x='day_of_week', y='Recalculated protein, %', kind='kde')

In [ ]:
data_fattening_without_500_run = data_fattening_without_500[['Diet_num','Larvae density',' total larval  biomass, gr','Average larval  mass, mg','Fat, %', 'Recalculated protein, %', 'Cycle, days','NDF, %', 'Ash, %', 'Protein, %', 'Carbs, without NDF \n%', 'Fat, \n%','Dry Matter, %','month','day_of_week']].dropna()

In [ ]:
grouped_df = data_fattening_without_500_run.groupby('Diet_num')['Fat, %'].describe()

grouped_df[grouped_df['std'] > 2]

In [ ]:
data_nursery_without_500.groupby('Diet_num')['Average larval  mass, mg'].describe()

### Running model on fattening data

In [ ]:
company_data = data_fattening_without_500_run

In [ ]:
#10 cross validation (k=10)
group_kfold = GroupKFold(n_splits=10)

X = company_data[['Diet_num','Ash, %', 'NDF, %','Carbs, without NDF \n%','Cycle, days','Protein, %','Fat, \n%','month','Larvae density','day_of_week']] #,'batch number ','Larvae density']]

y = company_data[[' total larval  biomass, gr','Average larval  mass, mg','Fat, %', 'Recalculated protein, %']]


In [ ]:
groups = company_data["Diet_num"]

In [ ]:
def run_fattening_model(xg_reg, groups, X,y):
    rmse_scores = []
    mae_scores = []
    me_scores = []

    one_hot_features = ['month','Larvae density','day_of_week']
    scale_features = [col for col in X.columns if col not in one_hot_features and col != 'Diet_num']

    preprocessor_X = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), scale_features),
            ('cat', 'passthrough', one_hot_features)
        ])

    pipeline_X = Pipeline(steps=[('preprocessor', preprocessor_X)])

    scaler_y = MinMaxScaler()

    for train_idx, test_idx in group_kfold.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


        print("Train diet numbers:", X_train['Diet_num'].unique())
        print("Test diet numbers:", X_test['Diet_num'].unique())

        X_train = X_train.drop('Diet_num', axis=1)
        X_test = X_test.drop('Diet_num', axis=1)


        X_train_transformed = pipeline_X.fit_transform(X_train)
        X_test_transformed = pipeline_X.transform(X_test)

        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test)

        model = MultiOutputRegressor(xg_reg)
        model.fit(X_train_transformed, y_train_scaled)

        y_pred_scaled = model.predict(X_test_transformed)
        y_pred = scaler_y.inverse_transform(y_pred_scaled)

        rmse_local_score = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
        mae_local_score = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
        mse_local_score = mean_squared_error(y_test, y_pred, multioutput='raw_values')
        me_score =  np.mean(y_test - y_pred)

        for i, target in enumerate(y_test.columns):
            print(f"{target} RMSE: {rmse_local_score[i]:.2f}")

        rmse_scores.append(rmse_local_score)
        mae_scores.append(mae_local_score)
        me_scores.append(me_score)

        print("-------------------------------")    


    avg_rmse_scores = np.mean(rmse_scores, axis=0)
    avg_mae_scores = np.mean(mae_scores, axis=0)
    avg_me_scores= np.mean(me_scores, axis=0)
    std_me_scores = np.std(me_scores, axis=0)


    for i, target in enumerate(y_test.columns):
        print(f"Average {target} RMSE: {avg_rmse_scores[i]:.2f}")
        print(f"Average {target} MAE: {avg_mae_scores[i]:.2f}")
        print(f"Average {target} ME: {avg_me_scores[i]:.2f}")

### Linear Regression for fattening

In [ ]:
xg_reg = LinearRegression() 

In [ ]:
run_fattening_model(xg_reg, groups, X,y)

### Random Forest

In [ ]:
xg_reg = RandomForestRegressor(n_estimators=100, random_state=42) 

In [ ]:
run_fattening_model(xg_reg, groups, X,y)

### XGBoost

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', seed=42)

In [ ]:
run_fattening_model(xg_reg, groups, X,y)

### Running model on nursery data

In [ ]:
data_nursery_without_500["Diet_num"].unique()

In [ ]:
company_data = data_nursery_without_500.copy()

In [ ]:
company_data[['Diet_num','NDF, %', 'Ash, %', 'Carbs, without NDF \n%','Protein, %','Fat, \n%','day_of_week','month','Larvae density']].dropna()

In [ ]:
group_kfold = GroupKFold(n_splits=10)

#Linear regression doesn't handle nan/missing values
company_data = company_data[['Diet_num','NDF, %', 'Ash, %', 'Carbs, without NDF \n%','Protein, %','Fat, \n%','day_of_week','month','Larvae density',' total larval  biomass, gr','Average larval  mass, mg']].dropna()

X = company_data[['Diet_num','NDF, %', 'Ash, %', 'Carbs, without NDF \n%','Protein, %','Fat, \n%','day_of_week','month','Larvae density']] #,'batch number ']]#,'Cycle, days']]
y = company_data[[' total larval  biomass, gr','Average larval  mass, mg']]

groups = company_data["Diet_num"]

In [ ]:
def run_nursery_model(xg_reg, groups, X,Y):
    rmse_scores = []
    mae_scores = []
    mse_scores = []
    me_scores = []

    one_hot_features = ['month','Larvae density','day_of_week']
    scale_features = [col for col in X.columns if col not in one_hot_features and col != 'Diet_num']

    preprocessor_X = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), scale_features),
            ('cat', 'passthrough', one_hot_features)
        ])

    pipeline_X = Pipeline(steps=[('preprocessor', preprocessor_X)])

    scaler_y = MinMaxScaler()

    for train_idx, test_idx in group_kfold.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


        print("Train diet numbers:", X_train['Diet_num'].unique())
        print("Test diet numbers:", X_test['Diet_num'].unique())

        X_train = X_train.drop('Diet_num', axis=1)
        X_test = X_test.drop('Diet_num', axis=1)

        X_train_transformed = pipeline_X.fit_transform(X_train)
        X_test_transformed = pipeline_X.transform(X_test)

        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test)

        model = MultiOutputRegressor(xg_reg)
        model.fit(X_train_transformed, y_train_scaled)

        y_pred_scaled = model.predict(X_test_transformed)
        y_pred = scaler_y.inverse_transform(y_pred_scaled)

        rmse_local_score = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
        mae_local_score = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
        mse_local_score = mean_squared_error(y_test, y_pred, multioutput='raw_values')
        me_local_score = np.mean(y_test - y_pred)

        for i, target in enumerate(y_test.columns):
            print(f"{target} RMSE: {rmse_local_score[i]:.2f}")


        rmse_scores.append(rmse_local_score)
        mse_scores.append(mse_local_score)
        mae_scores.append(mae_local_score)
        me_scores.append(me_local_score)

        print("-------------------------------")    

    avg_rmse_scores = np.mean(rmse_scores, axis=0)
    avg_mae_scores = np.mean(mae_scores, axis=0)
    avg_me_scores = np.mean(me_scores, axis=0)
    std_me_scores = np.std(me_scores, axis=0)


    for i, target in enumerate(y_test.columns):
        print(f"Average {target} RMSE: {avg_rmse_scores[i]:.2f}")
        print(f"Average {target} MAE: {avg_mae_scores[i]:.2f}")
        print(f"Average {target} ME: {avg_me_scores[i]:.2f}")

### Linear Regression

In [ ]:
xg_reg =LinearRegression() 

In [ ]:
run_nursery_model(xg_reg, groups, X,y)

### Random Forest for nursering

In [ ]:
xg_reg = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
run_nursery_model(xg_reg, groups, X,y)

### XGBoost for nursering

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', seed=42, learning_rate=0.1, max_depth=5, min_child_weight=1, gamma=0.0, colsample_bytree=0.3, n_estimators=100) 

In [ ]:
run_nursery_model(xg_reg, groups, X,y)